In [1]:
%reload_ext autoreload
%autoreload 3
%matplotlib inline

import os
import numpy
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt

# Return two paths from the current working directory
current = os.getcwd()
parent_1 = os.path.dirname(current)
parent_2 = os.path.dirname(parent_1)
root_dir = os.path.join(parent_2, 'interface')
# Add the root directory to the system path
os.sys.path.append(root_dir)
# H:\github\nnc-ufmg\micromap\src\micromap\interface\micromap_utils.py
from micromap_utils import MicroMAPReaderADS

In [2]:
def select_folder():
    # Create and force a foreground hidden root window
    root = tk.Tk()
    root.withdraw()
    root.lift()                  # Raise the window
    root.attributes('-topmost', True)  # Keep it on top

    # Show the folder picker
    folder = filedialog.askdirectory(title="Select the recording folder", parent=root)

    # Destroy everything cleanly
    root.update()
    root.destroy()

    if folder:
        print(f"Selected folder: {folder}")
    else:
        print("No folder selected.")

    return folder

In [3]:
data_folder = select_folder()
reader = MicroMAPReaderADS(data_folder)
reader.data.shape

Selected folder: C:/Users/mcjpe/Desktop/depois_lixo/square_2000_freq01_2025-04-30_18-29-19


(8, 38080)

In [4]:
def compute_fft(signal, sampling_rate):
    """
    Calcula a FFT de um sinal real e retorna apenas a metade positiva do espectro.

    Parâmetros:
    - signal: array 1D com o sinal no domínio do tempo
    - sampling_rate: taxa de amostragem em Hz

    Retorna:
    - freqs: vetor de frequências (em Hz)
    - fft_values: módulo da FFT correspondente
    """
    N = len(signal)
    fft_result = numpy.fft.fft(signal)
    fft_magnitude = numpy.abs(fft_result) / N  # Normalização

    freqs = numpy.fft.fftfreq(N, d=1/sampling_rate)

    # Mantém apenas a metade positiva do espectro (para sinais reais)
    half_N = N // 2
    return freqs[:half_N], fft_magnitude[:half_N]

In [6]:
%matplotlib qt

obj = reader
data = obj.data[:, 1550:]
rate = 2000#obj.sampling_freq
  
num_channels = data.shape[0]                                                                                       
time = numpy.arange(data.shape[1])                                                                                 # Creates a vector with the number of samples lenght
time = time /rate                                                                                                   # Converts the samples to seconds

fig = plt.figure(figsize=(10, 5))                                                                                 # Creates a figure with the size of 10x5 inches
ax_0 = fig.add_subplot(1, 1, 1)                                                                                  # Creates a 1x3 matrix of subplots

plot_channels = range(8)

fig, ax = plt.subplots(2, 1, figsize=(10, 5))                                                                      # Creates a figure with the size of 10x5 inches
ax[0].plot(time, data[0]*0.1e-3, color = (160/255, 17/255, 8/255, 1), linewidth = 1)                                          # Plots the first channel of the matrix

fft_f, fft_s = compute_fft(data[0], rate)                                                                        # Computes the FFT of the first channel 

ax[1].plot(fft_f, fft_s, color = (160/255, 17/255, 8/255, 1), linewidth = 1)                                 # Plots the FFT of the first channel


for ch in plot_channels:                                                                                             # Loop through all the rows of the matrix
    ax_0.plot(time, data[ch]*1e-4 + ch + 1, color = (160/255, 17/255, 8/255, 1), linewidth = 1)

ax_0.set_xlabel('TIME (s)', fontsize = 16)
ax_0.set_ylabel('CHANNEL', fontsize = 16)
ax_0.set_yticks(range(1, max(plot_channels) + 2, 2))
ax_0.set_yticklabels(range(1, max(plot_channels) + 2, 2))
ax_0.set_xlim(0, time[-1])

# set tick labels font
for label in (ax_0.get_xticklabels() + ax_0.get_yticklabels()):
    label.set_fontsize(16)

# plt.savefig(folders + '/signal_example.png')
# plt.savefig(folders + '/signal_example.svg')

plt.show()